In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.io import read_image

import timm
from timm.data import create_transform
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

import pandas as pd
import numpy as np
from PIL import Image
from tqdm.auto import tqdm

import os
import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# 대회 공지 평가 산식
def calc_puzzle(answer_df, submission_df):
    # Check for missing values in submission_df
    if submission_df.isnull().values.any():
        raise ValueError("The submission dataframe contains missing values.")

    # Public or Private answer Sample and Sorting by 'ID'
    submission_df = submission_df[submission_df.iloc[:, 0].isin(answer_df.iloc[:, 0])]
    submission_df = submission_df.sort_values(by='ID').reset_index(drop=True)

    # Check for length in submission_df
    if len(submission_df) != len(answer_df):
        raise ValueError("The submission dataframe wrong length.")

    # Convert position data to numpy arrays for efficient computation
    answer_positions = answer_df.iloc[:, 2:].to_numpy()  # Excluding ID, img_path, and type columns
    submission_positions = submission_df.iloc[:, 1:].to_numpy()  # Excluding ID column

    # Initialize the dictionary to hold accuracies
    accuracies = {}

    # Define combinations for 2x2 and 3x3 puzzles
    combinations_2x2 = [(i, j) for i in range(3) for j in range(3)]
    combinations_3x3 = [(i, j) for i in range(2) for j in range(2)]

    # 1x1 Puzzle Accuracy
    accuracies['1x1'] = np.mean(answer_positions == submission_positions)

    # Calculate accuracies for 2x2, 3x3, and 4x4 puzzles
    for size in range(2, 5):  # Loop through sizes 2, 3, 4
        correct_count = 0  # Initialize counter for correct full sub-puzzles
        total_subpuzzles = 0

        # Iterate through each sample's puzzle
        for i in range(len(answer_df)):
            puzzle_a = answer_positions[i].reshape(4, 4)
            puzzle_s = submission_positions[i].reshape(4, 4)
            combinations = combinations_2x2 if size == 2 else combinations_3x3 if size == 3 else [(0, 0)]

            # Calculate the number of correct sub-puzzles for this size within a 4x4
            for start_row, start_col in combinations:
                rows = slice(start_row, start_row + size)
                cols = slice(start_col, start_col + size)
                if np.array_equal(puzzle_a[rows, cols], puzzle_s[rows, cols]):
                    correct_count += 1
                total_subpuzzles += 1

        accuracies[f'{size}x{size}'] = correct_count / total_subpuzzles

    score = (accuracies['1x1'] + accuracies['2x2'] + accuracies['3x3'] + accuracies['4x4']) / 4.
    return score

In [3]:
# 사전학습된 모델 timm을 사용 
# timm중에서 deit3_base_patch16_384라는 모델을 사용한다.
# 24x24 패치로 나누기 때문에 4x4퍼즐을 맞춰도 영역이 일치하지 않는 문제가 발생하지 않습니다.
# 학습 자체는 기존 Jigsaw-Vit설정을 그대로 가져와서 사용
class Model(nn.Module):
    # 모델 생성 및 초기화
    # deit 모델의 일부 구성 요소를 변경하고, 선형 레이어를 추가
    # 모델 구성요소 변경 이외에 논문의 Jigsaw-ViT
    def __init__(self, mask_ratio = 0.0, pretrained = True):
        super().__init__()

        self.mask_ratio = mask_ratio
        self.pretrained = pretrained

        deit3 = timm.create_model('deit3_base_patch16_384', pretrained = pretrained)

        self.patch_embed = deit3.patch_embed
        self.cls_token = deit3.cls_token
        self.blocks = deit3.blocks
        self.norm = deit3.norm

        self.jigsaw = nn.Sequential(
            nn.Linear(768, 768),
            nn.ReLU(),
            nn.Linear(768, 768),
            nn.ReLU(),
            nn.Linear(768, 24*24)
        )

    # 이미지를 무작위로 마스킹하고, 마스킹된 위치의 인덱스를 반환
    # 논문의 random-masking부분
    def random_masking(self, x, mask_ratio):
        """
        Perform per-sample random masking by per-sample shuffling.
        Per-sample shuffling is done by argsort random noise.
        x: [N, L, D], sequence
        """
        N, L, D = x.shape  # batch, length, dim
        len_keep = int(L * (1 - mask_ratio))
    
        # 랜덤으로 노이즈를 생성, 노이즈 기준으로 정렬, 정렬된 인덱스 중에서 첫 부분을 선택하여 마스킹 적용x
        # 데이터의 일부를 임의로 마스킹하여 모델에 노이즈를 주어 훈련을 안정화하고 일반화 성능을 향상시키는데 사용가능
        noise = torch.rand(N, L, device=x.device)  # noise in [0, 1]

        # sort noise for each sample
        ids_shuffle = torch.argsort(noise, dim=1)  # ascend: small is keep, large is remove
        # target = einops.repeat(self.target, 'L -> N L', N=N)
        # target = target.to(x.device)

        # keep the first subset
        ids_keep = ids_shuffle[:, :len_keep] # N, len_keep
        x_masked = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, D))
        target_masked = ids_keep

        return x_masked, target_masked

    # 예측된 결과와 마스킹된 위치의 인덱스를 반환
    # 논문의 forward-jigsaw
    # github의 forward-jigsaw + forward-cls
    # 해당 부분이 MultiHead-Attention
    def forward(self, x):
        x = self.patch_embed(x)
        x, target = self.random_masking(x, self.mask_ratio)

        # append cls token
        cls_tokens = self.cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        # apply Transformer blocks
        x = self.blocks(x)
        x = self.norm(x)
        x = self.jigsaw(x[:, 1:])
        # -1은 크기 자동조절, 24x24의 사이즈로 자동 변환
        # target은 1차원 텐서로 자동 사이즈 조절
        return x.reshape(-1, 24*24), target.reshape(-1)

    # 예측된 결과를 반환
    # 논문의 forward부분
    # epochs 끝나고 실행
    def forward_test(self, x):
        x = self.patch_embed(x)

        # append cls token
        cls_tokens = self.cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        # apply Transformer blocks
        x = self.blocks(x)
        x = self.norm(x)
        x = self.jigsaw(x[:, 1:])
        # 패치의 값 576 => (384*384) / (16*16)
        return x

In [4]:
class JigsawDataset(Dataset):
    def __init__(self, df, data_path, mode='train', transform=None):
        self.df = df
        self.data_path = data_path
        self.mode = mode
        self.transform = transform

    def __len__(self):
        return len(self.df)

    # train일 경우
    # 이미지를 읽고, 1~16번째 순서 배열 생성
    # 이미지를 순서에 따라 재배치
    # 변환된 이미지 반환
    # test일 경우
    # 이미지를 읽고 변환이 정의되어 있다면 변환을 작용후 이미지 반환
    # transfrom은 build_transform을 통해 train, test 각각 생성
    def __getitem__(self, idx):
        if self.mode == 'train':
            row = self.df.iloc[idx]
            image = read_image(os.path.join(self.data_path, row['img_path']))
            shuffle_order = row[[str(i) for i in range(1, 17)]].values-1
            image = self.reset_image(image, shuffle_order)
            image = Image.fromarray(image)
            if self.transform:
                image = self.transform(image)
            return image
        elif self.mode == 'test':
            row = self.df.iloc[idx]
            image = Image.open(os.path.join(self.data_path, row['img_path']))
            if self.transform:
                image = self.transform(image)
            return image

    # 모델 입력을 위한 전처리 과정
    # 이미지를 순서에 따라 재배열
    def reset_image(self, image, shuffle_order):
        # 토치에서는 채널, 높이, 넓이로 구성되어있다.
        c, h, w = image.shape
        # 이미지를 4x4로 만들어야하기에 4로 나눈다.
        # 가로, 세로 길이를 4등분
        block_h, block_w = h//4, w//4
        # 4x4 배열의 초기화
        image_src = [[0 for _ in range(4)] for _ in range(4)]
        # order는 0~15
        for idx, order in enumerate(shuffle_order):
            h_idx, w_idx = divmod(order,4)
            h_idx_shuffle, w_idx_shuffle = divmod(idx, 4)
            image_src[h_idx][w_idx] = image[:, block_h * h_idx_shuffle : block_h * (h_idx_shuffle+1), block_w * w_idx_shuffle : block_w * (w_idx_shuffle+1)]
        # 한행의 패치를 가로로 연결, 그렇게 만들어진 가로줄을 세로로 연결
        image_src = np.concatenate([np.concatenate(image_row, -1) for image_row in image_src], -2)
        # 토치에서는 채널이 앞에 오니까 채널을 뒤로 보내서 사용가능하게 변환
        return image_src.transpose(1, 2, 0)

In [5]:
# github dataset/build-transform
def build_transform(is_train):
    # is_train이 True일 때, transfroms_imagenet_train을 이용해 데이터 증강을 적용
    if is_train:
        # this should always dispatch to transforms_imagenet_train
        # 매개변수를 통해, 색상, jittering, auto-argumentation을 설정
        # 384x384로 학습, 사전 학습 사용
        transform = create_transform(
            input_size = (384, 384),
            is_training = True,
            color_jitter = 0.3,
            auto_augment = 'rand-m9-mstd0.5-inc1',
            # 이미지 사이즈 변경시 주변 16개의 픽셀값을 계산해 이어지는 이미지의 곡선을 부드럽게 만든다.
            # 이미지의 세부사항을 유지하며 이미지의 사이즈를 바꿀 수 있다.
            interpolation= 'bicubic',
            re_prob= 0.25,
            re_mode= 'pixel',
            re_count= 1,
        )
        return transform
    
    # is_train이 False면 테스트 데이터에 대한 전처리 실행
    # 이미지 크기 조절(사용한 모델이 384사이즈 요구), 텐서 변환, 정규화 실행 후 반환
    # 마지막에 compose를 사용해 데이터셋에 대한 파이프라인 생성
    # = pytorch를 위한 파이프라인 생성
    t = []
    t.append(transforms.Resize((384,384), interpolation=3))
    t.append(transforms.ToTensor())
    t.append(transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD))
    return transforms.Compose(t)

In [6]:
df = pd.read_csv('./DATA/train.csv')

df = df.loc[:1999,:] # 수정상태 1만개는 메모리 초과 됨

In [46]:
test_1 = pd.read_csv("./data/test.csv")
test_2 = pd.read_csv("./data/sample_submission.csv")
test_df = test_1.merge(test_2, on='ID', how='inner')

test_df = test_df.loc[:3000,:]

In [47]:
train_df = df.iloc[:900] # 수정 상태
valid_df = df.iloc[900:] # 수정 상태
test_df = test_df.iloc[900:] # 수정 상태

train_transform = build_transform(is_train = True)
valid_transform = build_transform(is_train = False)
test_transform = build_transform(is_train = False)

train_dataset = JigsawDataset(df = train_df,
                              data_path = './DATA',
                              mode = 'train',
                              transform = train_transform)
valid_dataset = JigsawDataset(df = valid_df,
                              data_path = './DATA',
                              mode = 'test',
                              transform = valid_transform)

test_dataset = JigsawDataset(df = test_df,
                              data_path = './DATA',
                              mode = 'test',
                              transform = test_transform)

train_dataloader = DataLoader(
    train_dataset,
    batch_size = 16,
    shuffle = True
)
valid_dataloader = DataLoader(
    valid_dataset,
    batch_size = 16,
    shuffle = False
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size = 16,
    shuffle = False
)

In [9]:
model = Model(mask_ratio = 0.5)
model.to(device)
optimizer = optim.AdamW(model.parameters(),
                        lr=3e-5,
                        weight_decay = 0.05)

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
for epoch in range(1, 6):
    print('Epoch ', epoch)
    st = time.time()
    model.train()
    for i, x in enumerate(train_dataloader):
        x = x.to(device)

        optimizer.zero_grad()

        preds, targets = model(x)

        loss = F.cross_entropy(preds, targets)

        loss.backward()
        optimizer.step()

        if i % 100 == 0: # 수정상태
            print(f'[{i} / {len(train_dataloader)}] loss:', loss.item())
    et = time.time()
    print('Time elapsed: ', et-st)

Epoch  1
[0 / 57] loss: 6.37108039855957
Time elapsed:  1552.8252925872803
Epoch  2
[0 / 57] loss: 6.354735851287842
Time elapsed:  1621.489092350006
Epoch  3
[0 / 57] loss: 6.347467422485352
Time elapsed:  1585.6633050441742
Epoch  4
[0 / 57] loss: 6.308975696563721
Time elapsed:  1611.7624580860138
Epoch  5
[0 / 57] loss: 6.323413848876953
Time elapsed:  1627.0431520938873


In [11]:
outs = []
model.eval()
# 그라디언트 계산 비활성화로 메모리 절약
with torch.no_grad():
    for x in tqdm(valid_dataloader):
        x = x.to('cuda' if torch.cuda.is_available() else 'cpu') # 수정 상태
        # test데이터에 대한 예측 수행
        out = model.forward_test(x)
        # 각행 및 열에서 최댓값의 인덱스로 뱐환하고, gpu에서 cpu로 변환하는데 이번은 cpu로 계속했다.
        out = out.argmax(dim=2).cpu().numpy()
        outs.append(out)

# 예측된 결과를 수직으로 쌓는다.
outs = np.vstack(outs)
valid_pred_df = valid_df.copy().drop(columns=['img_path'])
# 얘측 결과를 기반으로 퍼즐을 복원하고, 그 결과를 검증 데이터 프레임에 업데이트
# I = total = len(valid_df)
# idx = 데이터 프레임의 인덱스
# row = 데이터 프레임의 1줄
for I, (idx, row) in enumerate(tqdm(valid_pred_df.iterrows(), total=len(valid_df))):
    # 24x24배열로 만듦
    w = outs[I].reshape(24,24)
    # 각 행과 열의 카운트를 저장할 배열을 초기화
    # 행, 열, 등장할 수 있는 값의 범위
    # 추후 해당위치에서 해당 값이 얼마나 나오는지 확인을 위함
    CNT_ROW = np.zeros((4,4,4), dtype=np.int32)
    CNT_COL = np.zeros((4,4,4), dtype=np.int32)
    # 퍼즐 순서 복원
    for i in range(24):
        for j in range(24):
            # 0~3
            ROW = i // 6
            COL = j // 6
            # 현 위치에서의 값 (현재는 24x24 배열이기 때문)
            v = w[i][j]
            # 세로방향이냐, 가로방향이냐에 따라 '//', '%'를 구분해서 사용
            # 행과 열에 대한 카운트에 대한 값 증가
            # 각 픽셀 값을 4x4로 만드는 과정
            # 해당위치에서의 값이 얼마나 자주 등장하는지 카운트 가능
            # 24x24 이미지를 4x4 퍼즐로 나누면 각 퍼즐은 6x6의 픽셀로 이루어집니다. 
            # 따라서 나누기 6을 통해 현재 위치의 픽셀이 해당 4x4 퍼즐에서 몇 번째 열에 속하는지를 나타낼 수 있습니다.
            # 24로 나눈 몫으로 세로 방향의 위치를 나타내며, 추가적인 // 6을 통해 해당 행 내에서의 상대적인 위치
            # 해당 위치의 width, height값이라서 나중에 합쳐준다.
            CNT_ROW[ROW][COL][v // 24 // 6] += 1
            # 열에 매핑하는 역할
            # 24로 나눈 나머지로 가로 방향의 위치를 나타내며, 추가적인 // 6을 통해 해당 열 내에서의 상대적인 위치
            CNT_COL[ROW][COL][v % 24 // 6] += 1
    # 각 행과 열에서 가장 많이 등장한 값으로 퍼즐을 복원합니다
    # 3차원 배열이라 argmax(2)
    # 0,1,2,3을 0,4,8,12로 만들어 주고 CNT_COL을 더하여 최종적으로 0~15 값으로 만들어준다.
    ans = CNT_ROW.argmax(2) * 4 + CNT_COL.argmax(2) + 1
    ans = ans.reshape(16)
    ans = list(map(str, ans))
    valid_pred_df.loc[idx, '1':'16'] = ans
score = calc_puzzle(valid_df, valid_pred_df)
print(score)

  0%|          | 0/69 [00:00<?, ?it/s]

  0%|          | 0/1100 [00:00<?, ?it/s]

0.0


In [12]:
valid_df.head()

,ID,img_path,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
900,TRAIN_00900,./train/TRAIN_00900.jpg,6,16,1,8,3,13,15,5,14,11,2,7,4,10,9,12
901,TRAIN_00901,./train/TRAIN_00901.jpg,10,8,14,15,1,16,3,7,12,5,9,11,4,6,2,13
902,TRAIN_00902,./train/TRAIN_00902.jpg,8,4,12,13,6,3,15,2,10,16,14,11,7,1,9,5
903,TRAIN_00903,./train/TRAIN_00903.jpg,3,15,13,4,11,14,12,1,6,10,16,9,2,5,7,8
904,TRAIN_00904,./train/TRAIN_00904.jpg,14,1,10,4,12,2,15,3,16,13,6,8,11,9,5,7


In [13]:
valid_pred_df.head()

,ID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
900,TRAIN_00900,5,1,5,11,1,1,1,5,1,6,1,5,1,1,5,6
901,TRAIN_00901,11,4,10,11,4,3,4,12,4,4,8,11,4,12,8,4
902,TRAIN_00902,4,5,6,6,11,1,4,4,6,8,6,11,11,4,2,4
903,TRAIN_00903,6,6,4,4,6,6,6,6,6,6,4,6,6,6,6,6
904,TRAIN_00904,6,4,6,6,11,6,4,4,4,6,6,6,6,6,6,6


In [14]:
model

Model(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inp

In [49]:
outs = []
model.eval()
with torch.no_grad():
    for x in tqdm(test_dataloader):
        x = x.to('cuda' if torch.cuda.is_available() else 'cpu') 
        out = model.forward_test(x)
        out = out.argmax(dim=2).cpu().numpy()
        outs.append(out)

outs = np.vstack(outs)
valid_pred_df = test_df.copy()
for I, (idx, row) in enumerate(tqdm(test_df.iterrows(), total=len(test_df))):
    w = outs[I].reshape(24,24)
    CNT_ROW = np.zeros((4,4,4), dtype=np.int32)
    CNT_COL = np.zeros((4,4,4), dtype=np.int32)
    for i in range(24):
        for j in range(24):
            ROW = i // 6
            COL = j // 6
            v = w[i][j]
            CNT_ROW[ROW][COL][v // 24 // 6] += 1
            CNT_COL[ROW][COL][v % 24 // 6] += 1
    ans = CNT_ROW.argmax(2) * 4 + CNT_COL.argmax(2) + 1
    ans = ans.reshape(16)
    ans = list(map(str, ans))
    valid_pred_df.loc[idx, '1':'16'] = ans

  0%|          | 0/132 [00:00<?, ?it/s]

  0%|          | 0/2101 [00:00<?, ?it/s]